# Integrate GDSC Drug Response with Methylation Data

**Goal**: Combine GSE68379 methylation data with GDSC drug response data to create a comprehensive pharmacoepigenomics dataset.

**Input**: 
- `data/processed/ML_dataset_methylation_features.csv.gz` (methylation only)
- `data/raw/GDSC1_fitted_dose_response_27Oct23.xlsx` (drug response)
- `data/raw/GDSC_Cell_Lines_Details.xlsx` (cell line annotations)

**Output**:
- `data/processed/ML_dataset_methylation_drug_response.csv.gz` (combined dataset)
- Updated documentation and metadata files

In [ ]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

# Display settings
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', 1000)

print("✓ Imports complete")

## 1. Load Existing Methylation Dataset

In [ ]:
# Load the methylation-only ML dataset we created previously
print("Loading methylation dataset...")
df_meth = pd.read_csv('../data/processed/ML_dataset_methylation_features.csv.gz',
                      index_col=0, compression='gzip')

print(f"Methylation dataset shape: {df_meth.shape}")
print(f"Cell lines: {df_meth.shape[0]:,}")
print(f"Total columns: {df_meth.shape[1]:,}")

# Identify column types
metadata_cols = ['primary site', 'primary histology', 'cosmic_id']
cpg_cols = [col for col in df_meth.columns if col.startswith('cg')]

print(f"\nColumn breakdown:")
print(f"  - Metadata: {len(metadata_cols)}")
print(f"  - CpG features: {len(cpg_cols):,}")

df_meth.head()

## 2. Load GDSC Drug Response Data

In [ ]:
# Load GDSC1 fitted dose response data
print("Loading GDSC drug response data...")
print("(This may take 30-60 seconds for the 28MB Excel file)\n")

df_gdsc_raw = pd.read_excel('../data/raw/GDSC1_fitted_dose_response_27Oct23.xlsx')

print(f"GDSC data loaded: {df_gdsc_raw.shape}")
print(f"Rows (drug-cell line combinations): {len(df_gdsc_raw):,}")
print(f"\nColumns:")
print(df_gdsc_raw.columns.tolist())

df_gdsc_raw.head(10)

In [ ]:
# Examine GDSC data structure
print("GDSC Data Overview:\n")
print(f"Unique cell lines: {df_gdsc_raw['CELL_LINE_NAME'].nunique():,}")
print(f"Unique drugs: {df_gdsc_raw['DRUG_NAME'].nunique()}")
print(f"Unique datasets: {df_gdsc_raw['DATASET'].unique() if 'DATASET' in df_gdsc_raw.columns else 'N/A'}")

print(f"\nKey columns for drug response:")
response_cols = [col for col in df_gdsc_raw.columns if any(x in col for x in ['IC50', 'AUC', 'RMSE', 'Z_SCORE'])]
for col in response_cols:
    print(f"  - {col}")

# Check data types
print(f"\nData types:")
print(df_gdsc_raw.dtypes)

In [ ]:
# Show sample drug response values
print("Sample drug response values:\n")
sample_cols = ['CELL_LINE_NAME', 'DRUG_NAME', 'LN_IC50', 'AUC', 'RMSE']
available_cols = [col for col in sample_cols if col in df_gdsc_raw.columns]
print(df_gdsc_raw[available_cols].head(20))

# Statistics for LN_IC50
if 'LN_IC50' in df_gdsc_raw.columns:
    print(f"\nLN_IC50 statistics:")
    print(df_gdsc_raw['LN_IC50'].describe())

## 3. Load GDSC Cell Line Annotations

In [ ]:
# Load cell line details for additional metadata
print("Loading GDSC cell line annotations...")
df_gdsc_cells = pd.read_excel('../data/raw/GDSC_Cell_Lines_Details.xlsx')

print(f"Cell line annotations: {df_gdsc_cells.shape}")
print(f"\nColumns:")
print(df_gdsc_cells.columns.tolist())

df_gdsc_cells.head()

## 4. Match Cell Lines Between GSE68379 and GDSC

Cell line names may differ between databases. We need to normalize and match them.

In [ ]:
# Get cell line names from both datasets
gse_cell_lines = set(df_meth.index)
gdsc_cell_lines = set(df_gdsc_raw['CELL_LINE_NAME'].unique())

print(f"GSE68379 cell lines: {len(gse_cell_lines):,}")
print(f"GDSC cell lines: {len(gdsc_cell_lines):,}")

# Find exact matches
exact_matches = gse_cell_lines & gdsc_cell_lines
print(f"\nExact name matches: {len(exact_matches)}")

if len(exact_matches) > 0:
    print(f"\nExamples of exact matches:")
    for name in list(exact_matches)[:10]:
        print(f"  - {name}")

In [ ]:
# Normalize cell line names for fuzzy matching
# Common variations: case, hyphens, underscores, spaces

def normalize_cell_line_name(name):
    """
    Normalize cell line name for matching.
    
    Transformations:
    - Convert to uppercase
    - Remove hyphens, underscores, spaces
    - Strip whitespace
    """
    return str(name).upper().replace('-', '').replace('_', '').replace(' ', '').strip()

# Create normalized mappings
gse_normalized = {normalize_cell_line_name(name): name for name in gse_cell_lines}
gdsc_normalized = {normalize_cell_line_name(name): name for name in gdsc_cell_lines}

print(f"GSE68379 normalized names: {len(gse_normalized)}")
print(f"GDSC normalized names: {len(gdsc_normalized)}")

# Find matches using normalized names
normalized_matches = set(gse_normalized.keys()) & set(gdsc_normalized.keys())
print(f"\nMatches after normalization: {len(normalized_matches)}")

# Create mapping dictionary: GDSC name -> GSE name
cell_line_mapping = {}
for norm_name in normalized_matches:
    gdsc_name = gdsc_normalized[norm_name]
    gse_name = gse_normalized[norm_name]
    cell_line_mapping[gdsc_name] = gse_name

print(f"\nCreated mapping for {len(cell_line_mapping)} cell lines")
print(f"\nExample mappings (GDSC -> GSE):")
for i, (gdsc_name, gse_name) in enumerate(list(cell_line_mapping.items())[:15]):
    match_type = "✓" if gdsc_name == gse_name else "→"
    print(f"  {match_type} {gdsc_name:20s} -> {gse_name}")

## 5. Filter and Process Drug Response Data

In [ ]:
# Filter GDSC data to matched cell lines only
print("Filtering GDSC data to matched cell lines...")

df_gdsc_filtered = df_gdsc_raw[
    df_gdsc_raw['CELL_LINE_NAME'].isin(cell_line_mapping.keys())
].copy()

print(f"GDSC entries before filtering: {len(df_gdsc_raw):,}")
print(f"GDSC entries after filtering: {len(df_gdsc_filtered):,}")
print(f"\nUnique cell lines (matched): {df_gdsc_filtered['CELL_LINE_NAME'].nunique()}")
print(f"Unique drugs: {df_gdsc_filtered['DRUG_NAME'].nunique()}")

# Map GDSC cell line names to GSE names
df_gdsc_filtered['GSE_CELL_LINE'] = df_gdsc_filtered['CELL_LINE_NAME'].map(cell_line_mapping)

print(f"\nSample of filtered data:")
df_gdsc_filtered[['CELL_LINE_NAME', 'GSE_CELL_LINE', 'DRUG_NAME', 'LN_IC50']].head(15)

In [ ]:
# Analyze drug coverage
print("Drug coverage analysis:\n")

drug_coverage = df_gdsc_filtered.groupby('DRUG_NAME').size().sort_values(ascending=False)

print(f"Total drugs: {len(drug_coverage)}")
print(f"\nDrug coverage statistics:")
print(drug_coverage.describe())

print(f"\nTop 20 drugs by number of cell lines tested:")
print(drug_coverage.head(20))

# Visualize coverage distribution
plt.figure(figsize=(10, 5))
plt.hist(drug_coverage.values, bins=50, edgecolor='black')
plt.axvline(100, color='red', linestyle='--', linewidth=2, label='100 cell lines (threshold)')
plt.xlabel('Number of Cell Lines Tested')
plt.ylabel('Number of Drugs')
plt.title('GDSC Drug Coverage Distribution')
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

# Count drugs above threshold
min_coverage = 100
high_coverage_drugs = drug_coverage[drug_coverage >= min_coverage]
print(f"\nDrugs tested in ≥{min_coverage} cell lines: {len(high_coverage_drugs)}")

In [ ]:
# Filter to high-quality drugs (≥100 cell lines tested)
print(f"Filtering to drugs tested in ≥{min_coverage} cell lines...")

high_quality_drugs = high_coverage_drugs.index.tolist()
df_gdsc_hq = df_gdsc_filtered[df_gdsc_filtered['DRUG_NAME'].isin(high_quality_drugs)].copy()

print(f"\nFiltered GDSC data:")
print(f"  Entries: {len(df_gdsc_hq):,}")
print(f"  Cell lines: {df_gdsc_hq['GSE_CELL_LINE'].nunique()}")
print(f"  Drugs: {df_gdsc_hq['DRUG_NAME'].nunique()}")

print(f"\nHigh-quality drugs ({len(high_quality_drugs)}):")
for i, drug in enumerate(high_quality_drugs[:20], 1):
    count = drug_coverage[drug]
    print(f"  {i:2d}. {drug:30s} ({count} cell lines)")
if len(high_quality_drugs) > 20:
    print(f"  ... and {len(high_quality_drugs) - 20} more drugs")

## 6. Create Drug Response Matrix

Pivot the data to create a matrix with:
- Rows: Cell lines
- Columns: Drugs
- Values: LN_IC50 (natural log of IC50)

In [ ]:
# Create drug response matrix
print("Creating drug response matrix...")

# Use LN_IC50 as the primary metric
# Lower LN_IC50 = more sensitive to drug (better response)
# Higher LN_IC50 = more resistant to drug (worse response)

df_drug_response = df_gdsc_hq.pivot_table(
    index='GSE_CELL_LINE',
    columns='DRUG_NAME',
    values='LN_IC50',
    aggfunc='mean'  # Average if multiple measurements exist
)

print(f"\nDrug response matrix shape: {df_drug_response.shape}")
print(f"  Cell lines (rows): {df_drug_response.shape[0]}")
print(f"  Drugs (columns): {df_drug_response.shape[1]}")

# Check missing data
missing_pct = df_drug_response.isna().sum().sum() / df_drug_response.size * 100
print(f"\nMissing data: {missing_pct:.2f}%")
print(f"  (Expected: not all drugs tested on all cell lines)")

# Show sample of data
print(f"\nSample drug response values (LN_IC50):")
df_drug_response.iloc[:10, :5]

In [ ]:
# Analyze drug response distribution
print("Drug response (LN_IC50) distribution:\n")
print(df_drug_response.stack().describe())

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Overall distribution
ax = axes[0]
df_drug_response.stack().hist(bins=50, ax=ax, edgecolor='black')
ax.set_xlabel('LN_IC50')
ax.set_ylabel('Frequency')
ax.set_title('Distribution of Drug Response (LN_IC50)\nLower = More Sensitive')
ax.axvline(0, color='red', linestyle='--', alpha=0.5, label='LN_IC50=0 (IC50=1µM)')
ax.legend()

# Missing data per drug
ax = axes[1]
missing_per_drug = df_drug_response.isna().sum().sort_values()
missing_per_drug.plot(kind='bar', ax=ax, color='steelblue')
ax.set_xlabel('Drug')
ax.set_ylabel('Missing Values')
ax.set_title('Missing Drug Response Data per Drug')
ax.set_xticklabels([])
ax.text(0.95, 0.95, f'Mean missing: {missing_per_drug.mean():.0f} cell lines',
        transform=ax.transAxes, ha='right', va='top',
        bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.tight_layout()
plt.savefig('../data/processed/drug_response_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"\n✓ Figure saved: data/processed/drug_response_distribution.png")

## 7. Combine Methylation and Drug Response Data

In [ ]:
# Find cell lines present in BOTH methylation and drug response datasets
print("Finding common cell lines...")

meth_cell_lines = set(df_meth.index)
drug_cell_lines = set(df_drug_response.index)
common_cell_lines = meth_cell_lines & drug_cell_lines

print(f"\nCell lines with methylation data: {len(meth_cell_lines)}")
print(f"Cell lines with drug response data: {len(drug_cell_lines)}")
print(f"Cell lines with BOTH: {len(common_cell_lines)}")

# Sort for consistent ordering
common_cell_lines = sorted(list(common_cell_lines))

In [ ]:
# Subset both datasets to common cell lines
print("Subsetting to common cell lines...")

df_meth_common = df_meth.loc[common_cell_lines].copy()
df_drug_common = df_drug_response.loc[common_cell_lines].copy()

print(f"\nMethylation data: {df_meth_common.shape}")
print(f"Drug response data: {df_drug_common.shape}")

# Verify indices match
assert (df_meth_common.index == df_drug_common.index).all(), "Index mismatch!"
print("\n✓ Indices verified to match")

In [ ]:
# Combine datasets
print("Combining methylation and drug response data...")

# Structure: [metadata] [methylation features] [drug targets]
df_combined = pd.concat([
    df_meth_common[metadata_cols],      # Metadata columns
    df_meth_common[cpg_cols],           # Methylation features
    df_drug_common                       # Drug response targets
], axis=1)

print(f"\nCombined dataset shape: {df_combined.shape}")
print(f"\nColumn structure:")
print(f"  1. Metadata: {len(metadata_cols)} columns")
print(f"  2. Methylation features: {len(cpg_cols):,} CpG sites")
print(f"  3. Drug response targets: {len(df_drug_common.columns)} drugs")
print(f"  Total: {df_combined.shape[1]:,} columns")

print(f"\nSample of combined dataset:")
df_combined.iloc[:5, [0, 1, 2, 3, 4, -3, -2, -1]]

## 8. Data Quality Assessment

In [ ]:
# Missing value analysis
print("Missing Value Analysis\n" + "="*50)

# Metadata
print("\n1. Metadata columns:")
metadata_missing = df_combined[metadata_cols].isna().sum()
print(metadata_missing)

# Methylation features
print("\n2. Methylation features:")
meth_missing_pct = df_combined[cpg_cols].isna().sum().sum() / (len(cpg_cols) * len(df_combined)) * 100
print(f"   Overall missing: {meth_missing_pct:.3f}%")
print(f"   (Essentially complete)")

# Drug response targets
print("\n3. Drug response targets:")
drug_cols = df_drug_common.columns.tolist()
drug_missing = df_combined[drug_cols].isna().sum()
drug_missing_pct = (drug_missing / len(df_combined) * 100).describe()

print(f"\n   Missing data per drug (% of cell lines):")
print(drug_missing_pct)

print(f"\n   Drugs with <10% missing: {(drug_missing < len(df_combined) * 0.1).sum()}")
print(f"   Drugs with <20% missing: {(drug_missing < len(df_combined) * 0.2).sum()}")
print(f"   Drugs with <30% missing: {(drug_missing < len(df_combined) * 0.3).sum()}")

In [ ]:
# Visualize data completeness
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Drug completeness
ax = axes[0]
drug_completeness_pct = (1 - drug_missing / len(df_combined)) * 100
drug_completeness_pct.hist(bins=30, ax=ax, edgecolor='black', color='steelblue')
ax.axvline(drug_completeness_pct.mean(), color='red', linestyle='--', linewidth=2,
           label=f'Mean: {drug_completeness_pct.mean():.1f}%')
ax.set_xlabel('Data Completeness (%)')
ax.set_ylabel('Number of Drugs')
ax.set_title('Drug Response Data Completeness\n(per Drug)')
ax.legend()
ax.grid(alpha=0.3)

# Cell line completeness
ax = axes[1]
cell_completeness = (1 - df_combined[drug_cols].isna().sum(axis=1) / len(drug_cols)) * 100
cell_completeness.hist(bins=30, ax=ax, edgecolor='black', color='coral')
ax.axvline(cell_completeness.mean(), color='red', linestyle='--', linewidth=2,
           label=f'Mean: {cell_completeness.mean():.1f}%')
ax.set_xlabel('Data Completeness (%)')
ax.set_ylabel('Number of Cell Lines')
ax.set_title('Drug Response Data Completeness\n(per Cell Line)')
ax.legend()
ax.grid(alpha=0.3)

plt.tight_layout()
plt.savefig('../data/processed/ML_dataset_completeness.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"\n✓ Figure saved: data/processed/ML_dataset_completeness.png")
print(f"\nCell line completeness summary:")
print(cell_completeness.describe())

## 9. Save Combined ML-Ready Dataset

In [ ]:
# Save combined dataset
output_file = Path('../data/processed/ML_dataset_methylation_drug_response.csv.gz')

print(f"Saving combined dataset to {output_file}...")
df_combined.to_csv(output_file, compression='gzip')

# Check file size
file_size_mb = output_file.stat().st_size / (1024 * 1024)
print(f"\n✓ File saved: {file_size_mb:.2f} MB")

# Also save methylation-only version for comparison
meth_only_file = Path('../data/processed/ML_dataset_methylation_features.csv.gz')
if meth_only_file.exists():
    meth_only_size = meth_only_file.stat().st_size / (1024 * 1024)
    print(f"\nComparison:")
    print(f"  Methylation only: {meth_only_size:.2f} MB")
    print(f"  With drug response: {file_size_mb:.2f} MB")
    print(f"  Size increase: {file_size_mb - meth_only_size:.2f} MB")

In [ ]:
# Create column information file
print("Creating column information file...")

column_info = pd.DataFrame({
    'column_name': df_combined.columns,
    'column_type': (
        ['metadata'] * len(metadata_cols) +
        ['methylation_feature'] * len(cpg_cols) +
        ['drug_response_target'] * len(drug_cols)
    ),
    'data_type': df_combined.dtypes.values.astype(str),
    'missing_count': df_combined.isna().sum().values,
    'missing_pct': (df_combined.isna().sum() / len(df_combined) * 100).values
})

column_info_file = Path('../data/processed/ML_dataset_column_info.csv')
column_info.to_csv(column_info_file, index=False)
print(f"✓ Column info saved: {column_info_file}")

print(f"\nColumn type breakdown:")
print(column_info['column_type'].value_counts())

print(f"\nSample column info:")
pd.concat([
    column_info[column_info['column_type'] == 'metadata'].head(3),
    column_info[column_info['column_type'] == 'methylation_feature'].head(5),
    column_info[column_info['column_type'] == 'drug_response_target'].head(10)
])

In [ ]:
# Create summary statistics file
print("Creating summary statistics file...")

summary_stats = {
    'dataset': 'GSE68379 (Methylation) + GDSC1 (Drug Response)',
    'created': pd.Timestamp.now().strftime('%Y-%m-%d'),
    'total_samples': len(df_combined),
    'total_features': len(cpg_cols),
    'total_drug_targets': len(drug_cols),
    'total_columns': df_combined.shape[1],
    'file_size_mb': round(file_size_mb, 2),
    'methylation_missing_pct': round(meth_missing_pct, 4),
    'drug_response_missing_pct_mean': round(drug_missing_pct['mean'], 2),
    'primary_sites': df_combined['primary site'].nunique(),
    'histologies': df_combined['primary histology'].nunique()
}

summary_file = Path('../data/processed/ML_dataset_summary.txt')
with open(summary_file, 'w') as f:
    f.write("ML-Ready Methylation + Drug Response Dataset Summary\n")
    f.write("=" * 50 + "\n\n")
    for key, value in summary_stats.items():
        f.write(f"{key:35s}: {value}\n")
    f.write("\n" + "=" * 50 + "\n\n")
    f.write("Dataset Structure:\n")
    f.write(f"  Rows (cell lines): {len(df_combined)}\n")
    f.write(f"  Columns:\n")
    f.write(f"    - Metadata: {len(metadata_cols)}\n")
    f.write(f"    - Methylation features: {len(cpg_cols):,} CpG sites\n")
    f.write(f"    - Drug response targets: {len(drug_cols)} drugs\n")
    f.write(f"\nUse Cases:\n")
    f.write(f"  - Predict drug response from DNA methylation\n")
    f.write(f"  - Identify pharmacoepigenomic biomarkers\n")
    f.write(f"  - Multi-task learning across drugs\n")
    f.write(f"  - Drug response classification (sensitive vs resistant)\n")

print(f"✓ Summary saved: {summary_file}")

print(f"\n" + "=" * 50)
print("Dataset Summary:")
print("=" * 50)
for key, value in summary_stats.items():
    print(f"{key:35s}: {value}")

## 10. Validation and Usage Example

In [ ]:
# Test loading the saved dataset
print("Testing dataset reload...")
df_test = pd.read_csv(output_file, compression='gzip', index_col=0)

print(f"✓ Dataset loaded successfully: {df_test.shape}")

# Verify integrity
assert df_test.shape == df_combined.shape, "Shape mismatch!"
assert (df_test.columns == df_combined.columns).all(), "Column mismatch!"
assert (df_test.index == df_combined.index).all(), "Index mismatch!"

print("✓ Dataset integrity verified")

In [ ]:
# Example usage for ML
print("\n" + "=" * 60)
print("EXAMPLE: Training a Drug Response Prediction Model")
print("=" * 60)

print("""
# 1. Load the dataset
import pandas as pd
df = pd.read_csv('data/processed/ML_dataset_methylation_drug_response.csv.gz',
                 index_col=0, compression='gzip')

# 2. Separate features and targets
metadata_cols = ['primary site', 'primary histology', 'cosmic_id']
methylation_features = [col for col in df.columns if col.startswith('cg')]
drug_targets = [col for col in df.columns 
                if col not in metadata_cols and not col.startswith('cg')]

X = df[methylation_features]  # Features: DNA methylation
y = df[drug_targets]          # Targets: Drug response (IC50)
metadata = df[metadata_cols]  # Sample information

# 3. Select a specific drug
drug_name = drug_targets[0]  # e.g., first drug in the list
y_drug = y[drug_name].dropna()  # Remove missing values

# Get corresponding samples
X_drug = X.loc[y_drug.index]

# 4. Train model
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

model = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42)
scores = cross_val_score(model, X_drug, y_drug, cv=5, 
                         scoring='neg_mean_squared_error')

print(f"Drug: {drug_name}")
print(f"Samples with data: {len(y_drug)}")
print(f"CV RMSE: {(-scores.mean())**0.5:.3f} (+/- {scores.std():.3f})")

# 5. Feature importance
model.fit(X_drug, y_drug)
feature_importance = pd.DataFrame({
    'CpG': X_drug.columns,
    'Importance': model.feature_importances_
}).sort_values('Importance', ascending=False)

print(f"\nTop 10 most important CpG sites:")
print(feature_importance.head(10))
""")

print(f"\nAvailable drugs for modeling: {len(drug_cols)}")
print(f"\nExample drug targets:")
for i, drug in enumerate(drug_cols[:15], 1):
    n_samples = df_combined[drug].notna().sum()
    print(f"  {i:2d}. {drug:30s} ({n_samples} samples)")
if len(drug_cols) > 15:
    print(f"  ... and {len(drug_cols) - 15} more drugs")

## Summary and Next Steps

In [ ]:
print("=" * 70)
print("GDSC DRUG RESPONSE INTEGRATION COMPLETE")
print("=" * 70)

print(f"\n📊 Dataset Statistics:")
print(f"  • Cell lines: {len(df_combined):,}")
print(f"  • Methylation features: {len(cpg_cols):,} CpG sites")
print(f"  • Drug targets: {len(drug_cols)} drugs")
print(f"  • Total columns: {df_combined.shape[1]:,}")
print(f"  • File size: {file_size_mb:.2f} MB")

print(f"\n📁 Output Files:")
print(f"  1. {output_file.name} - Main ML dataset")
print(f"  2. {column_info_file.name} - Column descriptions")
print(f"  3. {summary_file.name} - Summary statistics")
print(f"  4. drug_response_distribution.png - Drug response viz")
print(f"  5. ML_dataset_completeness.png - Data completeness viz")

print(f"\n🎯 Ready for Machine Learning!")
print(f"\nSuggested ML Tasks:")
print(f"  1. Single-drug response prediction (regression)")
print(f"  2. Multi-drug response prediction (multi-output)")
print(f"  3. Drug sensitivity classification (sensitive vs resistant)")
print(f"  4. Pharmacoepigenomic biomarker discovery")
print(f"  5. Cross-tissue drug response modeling")

print(f"\n🔬 Next Steps for Your Colleague:")
print(f"  • Start with high-coverage drugs (>200 samples)")
print(f"  • Use cross-validation to avoid overfitting")
print(f"  • Try RandomForest, XGBoost, or deep learning")
print(f"  • Extract feature importance to find biomarkers")
print(f"  • Consider tissue-stratified models")

print("\n" + "=" * 70)